Скрит позволяет разбивать изображения на части для обучения

In [17]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Параметры разбиения
PATCH_SIZE = 640
OVERLAP = 5
OUTPUT_DIR = "../DataSet/SplitDataAll/Valid/"
DATA_DIR = "../DataSet/NewDataAll/Valid/"


In [18]:
import os
import cv2

def split_image_and_labels(image_path, label_path, patch_size, overlap, output_dir):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    step = patch_size - overlap

    os.makedirs(os.path.join(output_dir, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels"), exist_ok=True)

    # Читаем метки (если файл пустой, то labels = [])
    labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:  # Фильтруем строки, где не 5 значений
                    labels.append([float(x) for x in parts])

    # Разрезаем изображение
    idx = 0
    for y in range(0, h, step):
        for x in range(0, w, step):
            x_end = min(x + patch_size, w)
            y_end = min(y + patch_size, h)
            patch = image[y:y_end, x:x_end]
            patch_h, patch_w, _ = patch.shape

            # Генерируем имя файла
            img_name = f"{os.path.splitext(os.path.basename(image_path))[0]}_{idx}.jpg"
            lbl_name = f"{os.path.splitext(os.path.basename(label_path))[0]}_{idx}.txt"

            cv2.imwrite(os.path.join(output_dir, "images", img_name), patch)

            # Собираем метки для этого патча
            patch_labels = []
            for label in labels:
                if len(label) == 5:  # Защита от некорректных данных
                    cls, x_center, y_center, width, height = label
                    x_abs = x_center * w
                    y_abs = y_center * h
                    box_w = width * w
                    box_h = height * h

                    # Проверяем, попадает ли объект в область
                    if x < x_abs < x_end and y < y_abs < y_end:
                        new_x_center = (x_abs - x) / patch_w
                        new_y_center = (y_abs - y) / patch_h
                        new_width = box_w / patch_w
                        new_height = box_h / patch_h
                        patch_labels.append([cls, new_x_center, new_y_center, new_width, new_height])

            # Сохраняем метки, если они есть
            label_output_path = os.path.join(output_dir, "labels", lbl_name)
            if patch_labels:  # Записываем только если есть метки
                with open(label_output_path, "w") as f:
                    for lbl in patch_labels:
                        f.write(" ".join(map(str, lbl)) + "\n")
            else:
                open(label_output_path, "w").close()  # Создаём пустой файл, если меток нет

            idx += 1


In [19]:

# Обработка всего набора данных

for img_name in tqdm(os.listdir(os.path.join(DATA_DIR, "images"))):
    img_path = os.path.join(DATA_DIR, "images", img_name)
    label_path = os.path.join(DATA_DIR, "labels", f"{os.path.splitext(img_name)[0]}.txt")

    # print(img_path, label_path)

    # with open(label_path, "r", encoding="utf-8") as f:
    #     labels = [line.strip().split() for line in f.readlines()]
    #     print("Содержимое labels:", labels)  # Посмотри, что реально загружено

    split_image_and_labels(
        image_path=img_path,
        label_path=label_path,
        patch_size=PATCH_SIZE,
        overlap=OVERLAP,
        output_dir=OUTPUT_DIR,
    )

100%|██████████| 1469/1469 [23:56<00:00,  1.02it/s]
